In [89]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interpolate
from PIL import Image

In [106]:
#Weather_SFO_2023.csv
#Weather_SLC_2023.csv
#Weather_HNL_2023.csv
#Weather_ANC_2023.csv
location = 'ANC'
filename = "../data/Weather_"+location+"_2023.csv"
df = pd.read_csv(filename, delimiter=',')
df.tail(5)

/var/folders/ww/rg8zlhq16mqg0kz1wk66r9b00000gn/T/ipykernel_5494/3146447271.py:7: DtypeWarning: Columns (21,27,28,29,31,32,33,35,42,44,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, delimiter=',')


,STATION,DATE,REPORT_TYPE,SOURCE,AWND,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupElevationUnit,BackupEquipment,BackupLatitude,BackupLongitude,BackupName,CDSD,CLDD,DSNW,DYHF,DYTS,DailyAverageDewPointTemperature,DailyAverageDryBulbTemperature,DailyAverageRelativeHumidity,DailyAverageSeaLevelPressure,DailyAverageStationPressure,DailyAverageWetBulbTemperature,DailyAverageWindSpeed,DailyCoolingDegreeDays,DailyDepartureFromNormalAverageTemperature,DailyHeatingDegreeDays,DailyMaximumDryBulbTemperature,DailyMinimumDryBulbTemperature,DailyPeakWindDirection,DailyPeakWindSpeed,DailyPrecipitation,DailySnowDepth,DailySnowfall,DailySustainedWindDirection,DailySustainedWindSpeed,DailyWeather,HDSD,HTDD,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,MonthlyAverageRH,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT32Temp,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithLT32Temp,MonthlyDepartureFromNormalAverageTemperature,MonthlyDepartureFromNormalCoolingDegreeDays,MonthlyDepartureFromNormalHeatingDegreeDays,MonthlyDepartureFromNormalMaximumTemperature,MonthlyDepartureFromNormalMinimumTemperature,MonthlyDepartureFromNormalPrecipitation,MonthlyDewpointTemperature,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureValueDate,MonthlyMaxSeaLevelPressureValueTime,MonthlyMaximumTemperature,MonthlyMeanTemperature,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureValueDate,MonthlyMinSeaLevelPressureValueTime,MonthlyMinimumTemperature,MonthlySeaLevelPressure,MonthlyStationPressure,MonthlyTotalLiquidPrecipitation,MonthlyTotalSnowfall,MonthlyWetBulb,NormalsCoolingDegreeDay,NormalsHeatingDegreeDay,REM,REPORT_TYPE.1,SOURCE.1,ShortDurationEndDate005,ShortDurationEndDate010,ShortDurationEndDate015,ShortDurationEndDate020,ShortDurationEndDate030,ShortDurationEndDate045,ShortDurationEndDate060,ShortDurationEndDate080,ShortDurationEndDate100,ShortDurationEndDate120,ShortDurationEndDate150,ShortDurationEndDate180,ShortDurationPrecipitationValue005,ShortDurationPrecipitationValue010,ShortDurationPrecipitationValue015,ShortDurationPrecipitationValue020,ShortDurationPrecipitationValue030,ShortDurationPrecipitationValue045,ShortDurationPrecipitationValue060,ShortDurationPrecipitationValue080,ShortDurationPrecipitationValue100,ShortDurationPrecipitationValue120,ShortDurationPrecipitationValue150,ShortDurationPrecipitationValue180,Sunrise,Sunset,WindEquipmentChangeDate
13020,70273026451,2023-12-21T21:00:00,FM-12,4,NaN,SE,1.7,mi,"TEMP, PRECIP, SNOW",131,NaN,"MMTS, SRG, SNOWBOARD, SNOWSTAKE",61.1561,-149.9847,"ANCHORAGE FORECAST OFFICE, AK",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,19,NaN,NaN,-0.04,3.0,88.0,29.44,15,29.26,9.94,18.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN08670273 32466 8//03 11072 21089 39909 4996...,FM-12,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-01
13021,70273026451,2023-12-21T21:53:00,FM-15,7,NaN,SE,1.7,mi,"TEMP, PRECIP, SNOW",131,NaN,"MMTS, SRG, SNOWBOARD, SNOWSTAKE",61.1561,-149.9847,"ANCHORAGE FORECAST OFFICE, AK",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.43,15,20,0.00,NaN,NaN,NaN,81.0,29.44,OVC:08 14,29.29,10.00,19.0,040,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [107]:
keep = ['DATE', 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyWindSpeed','HourlyRelativeHumidity', 'HourlySeaLevelPressure', 'HourlyWindDirection','HourlyVisibility']
df = df[keep]
print(df.columns.tolist())

['DATE', 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyWindSpeed', 'HourlyRelativeHumidity', 'HourlySeaLevelPressure', 'HourlyWindDirection', 'HourlyVisibility']


In [108]:
df = df.fillna(method='ffill')
df['timestamp']=pd.to_datetime(df['DATE'], format='%Y-%m-%dT%H:%M:%S')
df.set_index(df['timestamp'], inplace=True)
df.drop(columns=['DATE'], inplace=True)
# Drop the 'STATION' column
df.drop(columns=['timestamp'], inplace=True)
# Convert object columns to numeric
df = df.apply(pd.to_numeric, errors='coerce')
# Replace NaN values with 0.0
df.fillna(0.0, inplace=True)
print(df.dtypes)
df.head()

HourlyDryBulbTemperature    float64
HourlyPrecipitation         float64
HourlyWindSpeed             float64
HourlyRelativeHumidity      float64
HourlySeaLevelPressure      float64
HourlyWindDirection         float64
HourlyVisibility            float64
dtype: object


/var/folders/ww/rg8zlhq16mqg0kz1wk66r9b00000gn/T/ipykernel_5494/4122027356.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyVisibility
timestamp,,,,,,,
2022-12-21 00:11:00,-4.0,0.0,3.0,91.0,0.00,110.0,0.50
2022-12-21 00:53:00,-8.0,0.0,6.0,95.0,30.42,120.0,0.50
2022-12-21 01:22:00,-8.0,0.0,3.0,91.0,30.42,90.0,0.75
2022-12-21 01:53:00,-4.0,0.0,3.0,91.0,30.41,70.0,0.75
2022-12-21 02:53:00,-6.0,0.0,3.0,95.0,30.43,20.0,0.75


In [109]:
resampled_df = df.resample('60min').mean()
print(resampled_df.shape)
resampled_df.head()

(8784, 7)


,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyVisibility
timestamp,,,,,,,
2022-12-21 00:00:00,-6.0,0.0,4.5,93.000000,15.210,115.000000,0.500000
2022-12-21 01:00:00,-6.0,0.0,3.0,91.000000,30.415,80.000000,0.750000
2022-12-21 02:00:00,-6.0,0.0,3.0,95.000000,30.430,20.000000,0.750000
2022-12-21 03:00:00,-2.0,0.0,5.0,86.666667,30.430,243.333333,6.916667
2022-12-21 04:00:00,0.0,0.0,9.0,79.000000,30.440,340.000000,10.000000


In [110]:
start = pd.to_datetime('2022-12-21 22:00')
end = pd.to_datetime('2023-12-21 21:00')
df_2023 = resampled_df.loc[(resampled_df.index >= start) & (resampled_df.index <= end)]
df_2023.head(10)

,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyWindSpeed,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyWindDirection,HourlyVisibility
timestamp,,,,,,,
2022-12-21 22:00:00,13.0,0.0,9.0,67.0,30.410,360.0,10.00
2022-12-21 23:00:00,14.0,0.0,8.0,64.0,30.410,10.0,10.00
2022-12-22 00:00:00,15.0,0.0,9.0,61.0,30.400,10.0,10.00
2022-12-22 01:00:00,15.0,0.0,7.0,61.0,30.380,360.0,10.00
2022-12-22 02:00:00,16.0,0.0,8.0,62.0,30.370,10.0,10.00
2022-12-22 03:00:00,16.0,0.0,7.5,62.0,30.365,10.0,9.97
2022-12-22 04:00:00,16.0,0.0,7.0,65.0,30.330,360.0,10.00
2022-12-22 05:00:00,16.0,0.0,9.0,62.0,30.320,10.0,10.00
2022-12-22 06:00:00,16.0,0.0,9.0,62.0,30.290,10.0,10.00


In [111]:
df_2023.shape

(8760, 7)

In [112]:
df_2023.to_csv('../data/resampled_'+location+'_2023.csv',index=True)

In [113]:
# HourlyDryBulbTemperature    float64
# HourlyPrecipitation         float64
# HourlyWindSpeed             float64
# HourlyWindDirection         float64

## HourlyVisibility            float64
## HourlySeaLevelPressure      float64
## HourlyRelativeHumidity      float64
channel = ['HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyWindSpeed', 'HourlyWindDirection']
for ch in channel:
    
    # In this example, we directly use the values
    transformed_values = df_2023[ch]

    # Normalize the transformed values to the range 0-255
    min_val = transformed_values.min()
    max_val = transformed_values.max()
    normalized_values = ((transformed_values - min_val) / (max_val - min_val)) * 255

    # Replace non-finite values with 0.0
    normalized_values = np.nan_to_num(normalized_values, nan=0.0, posinf=0.0, neginf=0.0)
    normalized_values = normalized_values.astype(np.uint8)

    # Create an image
    height = len(normalized_values)
    image = Image.new('L', (1, height))  # 'L' mode for grayscale
    image.putdata(normalized_values)

    # Scale the image to 200 pixels wide
    scaled_image = image.resize((200, height))

    # Save the image
    image_filename = f'../img/{location}_{ch}_image_scaled.png'
    scaled_image.save(image_filename)

    # Optionally, display the image
    scaled_image.show()

    print(f'Saved image for {ch} to {image_filename}')


Saved image for HourlyDryBulbTemperature to ../img/ANC_HourlyDryBulbTemperature_image_scaled.png
Saved image for HourlyPrecipitation to ../img/ANC_HourlyPrecipitation_image_scaled.png
Saved image for HourlyWindSpeed to ../img/ANC_HourlyWindSpeed_image_scaled.png
Saved image for HourlyWindDirection to ../img/ANC_HourlyWindDirection_image_scaled.png
